# P0_Redd Carga de Datos, Limpieza y unión  
La base de datos se encuentra fraccionada en archivos de medida para cada dipositivo, y dos medidas de la red general de la casa.
Se uniran estos datos en un solo dataframe, con la finalidad de tener medidas ocurridas en el mismo instante de tiempo.  
Para esto se usara el metodo merge de la libreria pandas.  
Los nuevos dataframes se guardaran en nuevos archivos .csv para evitar este proceso en tareas futuras. 

In [253]:
#Librerias 
import pandas as pd
import os

In [254]:
#Ruta de la carpeta base
wdirectory = os.getcwd()
wdirectory = wdirectory + '\low_freq'
print(f'{wdirectory}')
list_dir = os.listdir(wdirectory)
print(list_dir)

C:\Users\josel\REDD_Pruebas\low_freq
['.ipynb_checkpoints', 'casas.csv', 'casasJRus.pkl', 'house_1', 'house_2', 'house_3', 'house_4', 'house_5', 'house_6']


In [255]:
#Casas en low_freq
casas = [casa for casa in list_dir if casa.find('house_',0,len(casa)) != -1]
print(casas)

['house_1', 'house_2', 'house_3', 'house_4', 'house_5', 'house_6']


In [256]:
#Guardar direcciones de cada casa
dir_casas = [os.path.join(wdirectory, casa) for casa in casas]
print(dir_casas)

['C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_1', 'C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_2', 'C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_3', 'C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_4', 'C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_5', 'C:\\Users\\josel\\REDD_Pruebas\\low_freq\\house_6']


In [257]:
#Guardar direcciones de los dispositivos de cada casa
def directorio_dispositivos(casa):
    #Direccion de casa
    list_devices = os.listdir(dir_canales[casa-1])
    max = len(list_devices)
    dir_devices = [os.path.join(dir_canales[casa-1], "channel_"+str(i) + ".dat") for i in range(1,max)]
    return dir_devices
casa_1 = lista_dispositivos(1)
casa_2 = lista_dispositivos(2)
casa_3 = lista_dispositivos(3)
casa_4 = lista_dispositivos(4)
casa_5 = lista_dispositivos(5)
casa_6 = lista_dispositivos(6)

In [258]:
#Funcion que lee los nombres de dispositivos en cada casa
def nombre_dispositivos(casa):
    #direccion de casa
    dir_casa = (dir_canales[casa-1]) + "\\labels.dat"
    #Diccionario con nombres dispositivos
    label = {}
    #Carga el directorio
    with open(dir_casa) as f:
        #Lee el archivo
        for line in f:
            #Separo las palabras
            splitted_line = line.split(' ')
            #Guarda las palabras encontradas usando el indice como key y el nombre como value
            label[int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
    #Retorna el diccionario con las etiquetas
    return label
labels_casa_1 = nombre_dispositivos(1)
labels_casa_2 = nombre_dispositivos(2)
labels_casa_3 = nombre_dispositivos(3)
labels_casa_4 = nombre_dispositivos(4)
labels_casa_5 = nombre_dispositivos(5)
labels_casa_6 = nombre_dispositivos(6)

In [259]:
labels_casa_6

{1: 'mains_1',
 2: 'mains_2',
 3: 'kitchen_outlets_3',
 4: 'washer_dryer_4',
 5: 'stove_5',
 6: 'electronics_6',
 7: 'bathroom_gfi_7',
 8: 'refrigerator_8',
 9: 'dishwaser_9',
 10: 'outlets_unknown_10',
 11: 'outlets_unknown_11',
 12: 'electric_heat_12',
 13: 'kitchen_outlets_13',
 14: 'lighting_14',
 15: 'air_conditioning_15',
 16: 'air_conditioning_16',
 17: 'air_conditioning_17'}

## Uniendo Dataframes
https://www.analyticslane.com/2018/09/10/unir-y-combinar-dataframes-con-pandas-en-python/

In [260]:
#Prueba para una sola casa unir dos canales de casa 1
#channel_1 = pd.read_csv(casa_1[0],sep=r' ',encoding='utf-8',names=['UTC_Unix',labels_casa_1[1]],header=None)
#channel_2 = pd.read_csv(casa_1[0],sep=r' ',encoding='utf-8',names=['UTC_Unix',labels_casa_1[2]],header=None)
#Unir dos dataframes usando como clave el tiempo Unix
#nuevo = pd.merge(channel_1, channel_2, how = 'inner', on = 'UTC_Unix')
#print(nuevo)

In [261]:
#Función para leer y unir dataframes
def unir_data(casa, label_casa):

    #Cargo la primera columna para tener un valor inicial#
    channel_inicial = pd.read_csv(casa[0],sep=r' ',encoding='utf-8',names=['UTC_Unix',label_casa[1]],header=None)

    num_devices = len(label_casa)
    for device in range(1, num_devices):

        channel_A = pd.read_csv(casa[device],sep=r' ',encoding='utf-8',names=['UTC_Unix',label_casa[device+1]],header=None)

        channel_inicial = pd.merge(channel_inicial, channel_A,  how = 'inner', on = 'UTC_Unix')

        if device == num_devices-1:
            #Modificar el formato Utc_time a fecha
            channel_inicial["UTC_Unix"] = channel_inicial['UTC_Unix'].astype("datetime64[s]")
            #Usar el tiempo como indice del dataset
            channel_inicial = channel_inicial.set_index(channel_inicial['UTC_Unix'].values)
            #Eliminado el tiempo como columna de los datos
            channel_inicial.drop(["UTC_Unix"], axis=1, inplace=True)
            return channel_inicial
            break        

Creando los conjuntos para las 6 casas

In [262]:
data_casa_1 = unir_data(casa_1, labels_casa_1)
data_casa_2 = unir_data(casa_2, labels_casa_2)
data_casa_3 = unir_data(casa_3, labels_casa_3)
data_casa_4 = unir_data(casa_4, labels_casa_4)
data_casa_5 = unir_data(casa_5, labels_casa_5)
data_casa_6 = unir_data(casa_6, labels_casa_6)

In [263]:
print(f'Casa 1: {type(data_casa_1)}, {data_casa_1.shape}')
print(f'Casa 2: {type(data_casa_2)}, {data_casa_2.shape}')
print(f'Casa 3: {type(data_casa_3)}, {data_casa_3.shape}')
print(f'Casa 4: {type(data_casa_4)}, {data_casa_4.shape}')
print(f'Casa 5: {type(data_casa_5)}, {data_casa_5.shape}')
print(f'Casa 6: {type(data_casa_6)}, {data_casa_6.shape}')

Casa 1: <class 'pandas.core.frame.DataFrame'>, (406748, 20)
Casa 2: <class 'pandas.core.frame.DataFrame'>, (316840, 11)
Casa 3: <class 'pandas.core.frame.DataFrame'>, (376150, 22)
Casa 4: <class 'pandas.core.frame.DataFrame'>, (428076, 20)
Casa 5: <class 'pandas.core.frame.DataFrame'>, (77451, 26)
Casa 6: <class 'pandas.core.frame.DataFrame'>, (192192, 17)


## Verificando si hay valores NaN
En este caso estos valores los tomaremos como 0.  
La siguiente función entrega el número de valores NaN en el dataframe

In [264]:
print(data_casa_1.isnull().sum().sum())
print(data_casa_2.isnull().sum().sum())
print(data_casa_3.isnull().sum().sum())
print(data_casa_4.isnull().sum().sum())
print(data_casa_5.isnull().sum().sum())
print(data_casa_6.isnull().sum().sum())

0
0
0
0
0
0


En caso de encontrar alguno lo remplazamos por 0.0, en esta oportunidad no los hay. La siguiente linea de código se usaria para este fin. 

In [275]:
#data_casa_1.fillna(0.0)

,mains_1,mains_2,oven_3,oven_4,refrigerator_5,dishwaser_6,kitchen_outlets_7,kitchen_outlets_8,lighting_9,washer_dryer_10,microwave_11,bathroom_gfi_12,electric_heat_13,stove_14,kitchen_outlets_15,kitchen_outlets_16,lighting_17,lighting_18,washer_dryer_19,washer_dryer_20
2011-04-18 13:22:13,222.20,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:16,223.17,119.19,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:20,223.60,118.92,0.0,0.0,6.0,0.0,34.0,22.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:23,222.91,119.16,0.0,0.0,6.0,1.0,35.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:26,222.94,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-05-24 19:56:20,235.73,38.65,0.0,0.0,186.0,1.0,26.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:23,235.03,38.66,0.0,0.0,187.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:27,235.46,38.61,0.0,0.0,190.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:30,235.98,38.77,0.0,0.0,189.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## Guardar conjuntos de datos
Se hace para evitar el poceso de Unión ya que consume tiempo  
https://recursospython.com/guias-y-manuales/os-shutil-archivos-carpetas/

In [265]:
#Crear carpeta para los archivos condensados de cada casa
new_directory = os.getcwd()
#print(new_directory + "\\low_freq\\data_casas")
os.makedirs(new_directory + "\\low_freq\\data_casas")

In [266]:
#os.removedirs(new_directory + "\\low_freq\\data_casas")

Guardando los datos en formato csv con encabezado e indice  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [267]:
data_casa_1.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_1.csv", index=True,sep=r' ',header=True, encoding='utf-8')
data_casa_2.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_2.csv", index=True,sep=r' ',header=True, encoding='utf-8')
data_casa_3.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_3.csv", index=True,sep=r' ',header=True, encoding='utf-8')
data_casa_4.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_4.csv", index=True,sep=r' ',header=True, encoding='utf-8')
data_casa_5.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_5.csv", index=True,sep=r' ',header=True, encoding='utf-8')
data_casa_6.to_csv(new_directory + "\\low_freq\\data_casas\\data_casa_6.csv", index=True,sep=r' ',header=True, encoding='utf-8')

Probando que los archivos se guardaron de manera correcta y conservan la información

In [268]:
data_casa_1 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_1.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)
data_casa_2 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_2.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)
data_casa_3 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_3.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)
data_casa_4 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_4.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)
data_casa_5 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_5.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)
data_casa_6 = pd.read_csv(new_directory + "\\low_freq\\data_casas\\data_casa_6.csv",sep=r' ',encoding='utf-8',header=0, index_col = 0)


In [276]:
data_casa_1

,mains_1,mains_2,oven_3,oven_4,refrigerator_5,dishwaser_6,kitchen_outlets_7,kitchen_outlets_8,lighting_9,washer_dryer_10,microwave_11,bathroom_gfi_12,electric_heat_13,stove_14,kitchen_outlets_15,kitchen_outlets_16,lighting_17,lighting_18,washer_dryer_19,washer_dryer_20
2011-04-18 13:22:13,222.20,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:16,223.17,119.19,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:20,223.60,118.92,0.0,0.0,6.0,0.0,34.0,22.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:23,222.91,119.16,0.0,0.0,6.0,1.0,35.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0
2011-04-18 13:22:26,222.94,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,0.0,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-05-24 19:56:20,235.73,38.65,0.0,0.0,186.0,1.0,26.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:23,235.03,38.66,0.0,0.0,187.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:27,235.46,38.61,0.0,0.0,190.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2011-05-24 19:56:30,235.98,38.77,0.0,0.0,189.0,0.0,24.0,20.0,2.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Análisis de datos
https://datacarpentry.org/python-ecology-lesson-es/02-starting-with-data/